<a href="https://colab.research.google.com/github/lilxmx/DeepLearning-pytorch/blob/main/neural_networks_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline


神经网络
===============

神经网络通过``torch.nn``构造.

现在你已经对``autograd`` 有了一个大致的了解, ``nn`` 依赖 ``autograd``去定义模型并对其参数进行微分。
一个模型``nn.Module`` 包含很多层，一个方法 ``forward(input)`` 可以返回模型结果 ``output``.

例如，用于手写数字图像分类的网络模型
![network that classifies digit images](https://pytorch.org/tutorials/_images/mnist.png)

它是一个简单的前馈神经网络，获取输入，一层一层向后传递，最终获取输出值。

一个神经网络经典训练流程如下:

- 定义神经网络的可学习的参数（或权重weights）
- 迭代输入数据集
- 将输入数据传入网络
- 计算损失函数（正确值和预测值相差多少）
- 对网络的参数进行反向传播
- 更新网络的权重，通常使用一个简单的更新规则:
  ``weight = weight - learning_rate * gradient``

定义网络结构
------------------

我们一起来定义一个网络结构吧：



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension 
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


你只定义了前向传播（``forward``）函数, 用于计算梯度的反向传播（``backward``）函数已经自动定义了。  
你可以使用任意Tensor操作（operations）在``forward``函数中  
``net.parameters()``方法可以返回一个模型可学习的参数




In [ ]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


让我们随机生成一个32x32的输入数据来测试这个网络吧！  
**注意**: LeNet神经网络允许的输入数据形状是 32x32。 在此网络上使用MNIST数据集，请重新设置数据集中图片的形状为 32x32.



In [ ]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.2039,  0.0686,  0.0305,  0.1299, -0.1100, -0.0012, -0.0745,  0.1204,
          0.0217,  0.0494]], grad_fn=<AddmmBackward0>)


将所有参数的梯度缓存清零，然后进行随机梯度的的反向传播：



In [ ]:
net.zero_grad()
out.backward(torch.randn(1, 10))

#### 注意：  
``torch.nn``仅支持输入数据为mini-batches类型，不能直接使用数据集的全部样本或者单个样本进行训练.  
例如，``nn.Conv2d`` 需要输入数据为4维张量，其形状为：``（nSamples x nChannels x Height x Width）``。  
 如果你有单个样本，只需使用 ``input.unsqueeze(0)`` 来添加其它的维数

在继续之前，我们回顾一下到目前为止用到的类。

**回顾:**
  -  ``torch.Tensor`` - 一个支持自动求导操作（``backward()）的多维数组A，存储者张量的梯度值。
  -  ``nn.Module`` - 神经网络模块，提供了一种封装网络参数的便捷方式，有用于将参数移动到 GPU、导出、加载等的助手。
  -  ``nn.Parameter`` - 一种变量，当把它赋值给一个Module时，被 自动 地注册为一个参数。
  -  ``autograd.Function`` - 实现一个自动求导操作的前向和反向定义，每个变量操作至少创建一个函数节点，每一个Tensor的操作都回创建一个接到创建Tensor和 编码其历史的函数的Function节点。

**已经介绍了：**
  -  定义一个神经网络
  -  处理输入，反向传播

**还剩:**
  -  计算损失函数
  -  更新网络权重

损失函数
-------------
一个损失函数接受一对 (output, target) 作为输入，计算一个值来估计网络的输出和目标值相差多少。  
译者注：output为网络的输出，target为实际值

nn包中有很多[不同的损失函数](https://pytorch.org/docs/stable/nn.html)。 nn.MSELoss是一个比较简单的损失函数，它计算输出和目标间的均方误差， 例如：



In [ ]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.0551, grad_fn=<MseLossBackward0>)


现在，如果在反向过程中跟随loss ， 使用它的 .grad_fn 属性，将看到如下所示的计算图。
::

    input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
          -> flatten -> linear -> relu -> linear -> relu -> linear
          -> MSELoss
          -> loss

所以，当我们调用 loss.backward()时,整张计算图都会 根据loss进行微分，而且图中所有设置为requires_grad=True的张量 将会拥有一个随着梯度累积的.grad 张量。

为了更好的解释这个现象，后退几步。


In [ ]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

反向传播
--------
调用`loss.backward()`获得反向传播的误差。

但是在调用前需要清除已存在的梯度，否则梯度将被累加到已存在的梯度。

现在，我们将调用`loss.backward()`，并查看conv1层的偏差（bias）项在反向传播前后的梯度。


In [ ]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0228,  0.0096,  0.0050,  0.0078, -0.0029,  0.0002])


Now, we have seen how to use loss functions.

稍后阅读：

`nn`包，包含了各种用来构成深度神经网络构建块的模块和损失函数，完整的文档请查看[here](https://pytorch.org/docs/stable/nn.html)。

剩下的最后一件事:

- 更新网络的权重

在实践中最简单的权重更新规则是随机梯度下降（SGD）：  

 ``weight = weight - learning_rate * gradient``  
我们可以使用简单的Python代码实现这个规则：

.. code:: python

    learning_rate = 0.01
    for f in net.parameters():
        f.data.sub_(f.grad.data * learning_rate)

但是当使用神经网络是想要使用各种不同的更新规则时，比如SGD、Nesterov-SGD、Adam、RMSPROP等，PyTorch中构建了一个包torch.optim实现了所有的这些规则。 使用它们非常简单：



In [10]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

#### 注意：

  观察如何使用``optimizer.zero_grad()``手动将梯度缓冲区设置为零。
  这是因为梯度是按Backprop部分中的说明累积的

